In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK：用于在线预测的自定义训练图像分类模型，可解释性

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_image_classification_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexplainable_ai%2Fsdk_custom_image_classification_online_explain.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/sdk_custom_image_classification_online_explain.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI SDK训练和部署自定义图像分类模型，以便进行带解释的在线预测。

了解有关[Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)和[Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions)的更多信息。

###目标

在本教程中，您将学习如何使用Vertex AI训练和可解释AI来创建具有解释的自定义图像分类模型。然后，您将学习如何使用Vertex AI在线预测服务发出带有解释的在线预测请求。或者，您可以使用`gcloud`命令行工具或在线使用Cloud Console创建自定义模型。

本教程使用以下Vertex AI服务：

- Vertex AI训练
- Vertex AI在线预测
- Vertex可解释AI
- Vertex AI模型资源
- Vertex AI端点资源

执行的步骤包括：

- 为训练TensorFlow模型创建Vertex AI自定义作业。
- 查看训练模型的模型评估。
- 设置模型部署时的解释参数。
- 上载训练的模型工件和解释作为模型资源。
- 创建一个服务端点资源。
- 将模型资源部署到服务端点资源。
- 进行带有解释的预测。
- 取消部署模型资源。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您所使用的数据集版本已集成到TensorFlow中。训练好的模型会预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

费用

本教程使用谷歌云的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成费用估算。

开始吧

### 为Python安装Vertex AI SDK和其他所需的软件包

### 安装Vertex AI SDK和其他必需的软件包

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform
! pip3 install --upgrade --quiet google-cloud-storage
! pip3 install --upgrade --quiet opencv-python
! pip3 install --upgrade --quiet matplotlib
! pip3 install --quiet tensorflow==2.15.1

if os.getenv("IS_TESTING"):
    ! apt-get update && apt-get install -y python3-opencv-headless
    ! apt-get install -y libgl1-mesa-dev
    ! pip3 install --upgrade opencv-python-headless -quiet

### 重新启动运行时（仅限 Colab）

为了使用新安装的软件包，您必须在 Google Colab 上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核即将重新启动。请等待完成后再继续下一步。⚠️</b>
</div>

验证您的笔记本环境（仅限Colab）

在谷歌Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储存储桶

创建一个存储桶来存储诸如数据集等中间产物。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入所需的库

In [ ]:
import base64
import io
import os
from io import BytesIO

import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from google.cloud import aiplatform
from tensorflow.keras.datasets import cifar10

### 初始化 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

使用位置和云存储桶为您的项目初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 以使用支持 GPU 的容器映像，并分配给虚拟机实例（VM）的 GPU 数量。例如，要使用一个包含 4 个 Nvidia Telsa T4 GPU 的 GPU 容器映像，分配给每个 VM，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，指定 `(None, None)` 使用一个 CPU 运行的容器映像。

了解更多关于 [硬件加速器在您所在地区的支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

**注意**：在 TF 2.3 之前发布的版本中，由于在服务函数中生成了静态图操作，可能导致在本教程中无法加载自定义模型的 GPU 支持。这是一个已知问题，在 TF 2.3 中已修复。如果您在自己的自定义模型上遇到此问题，请使用具有 GPU 支持的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置用于训练和预测的预先构建的Docker容器映像。

有关最新列表，请参见[用于训练的预建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参见[用于预测的预建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-11"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的虚拟机的计算资源。
- `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
- `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

**注意**：以下内容不支持用于训练：

- `standard`：2个vCPUs
- `highcpu`：2、4和8个vCPUs

**注意**：您也可以在训练和部署时使用n2和e2机器类型，但它们不支持GPU。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

检查培训套件

现在您已经准备好开始创建自己的自定义模型，并对CIFAR10进行训练。

在开始训练之前，请查看下面的训练应用程序包。

### 包布局

在开始训练之前，请查看如何将Python包装为自定义训练任务。解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`包含将软件包安装到Docker镜像的操作环境的说明。

文件`trainer/task.py`是执行自定义训练任务的Python脚本。

**注意**：在引用工作池规范时，目录斜杠被替换为点（`trainer.task`），文件后缀（`.py`）被删除。

### 包装配件

在以下单元格中，您将装配训练包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Task.py 内容

在下一个单元格中，您将编写训练脚本 *task.py* 的内容。

总的来说，*task.py* 脚本执行以下操作：

- 从命令行获取保存模型文件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF 数据集（tfds）中加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 根据参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将经过训练的模型保存到指定的模型目录（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

## 将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_cifar10.tar.gz

## 创建并运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练作业，2）运行这个作业。

### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，需要提供以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="cifar10",
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的训练参数

现在为您的自定义训练容器定义命令行参数：

- `args`：传递给将作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：用于指定存储模型工件的位置的命令行参数。您可以使用以下任一方法指定工件的存储位置。
      - **方法1**（将`DIRECT`设置为`True`）：您将Cloud Storage位置作为命令行参数传递给您的训练脚本。
      - **方法2**（将`DIRECT`设置为`False`）：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本。在这种情况下，您在作业规范中告诉服务模型工件的位置。
  - `--epochs`：训练的时代数。
  - `--steps`：每个时代的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, "model")

EPOCHS = 20
STEPS = 100

DIRECT = True  # Set False to use AIP_MODEL_DIR
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

### 运行自定义训练作业

接下来，您可以运行自定义作业以通过调用 `run()` 方法开始训练作业，以下是参数：

- `args`: 要传递给训练脚本的命令行参数。
- `replica_count`: 用于训练的计算实例数量（replica_count = 1 代表单节点训练）。
- `machine_type`: 计算实例的机器类型。
- `accelerator_type`: 硬件加速器类型。
- `accelerator_count`: 要附加到工作人员副本的加速器数量。
- `base_output_dir`: 用于存储模型工件的 Cloud Storage 位置。
- `sync`: 设置为 **True** 以等待作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

## 加载已保存的模型

您的模型以TensorFlow SavedModel格式存储在Cloud Storage存储桶中。现在从Cloud Storage存储桶中加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载模型，您可以使用TF.Keras的`model.load_model()`方法，将其传递给模型保存的Cloud Storage路径 -- 由`MODEL_DIR`指定。

In [ ]:
local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在，找出模型的表现如何。准备一些用于评估的测试数据，并运行`evaluate()`方法。

### 加载评估数据

使用`tf.keras.datasets`中的`load_data()`方法加载CIFAR10测试（留存）数据。这会返回一个包含两个元素的元组，第一个元素是训练数据，第二个元素是测试数据。每个元素也是一个包含两个元素的元组：图像数据和对应的标签。

对于这一步，你不需要训练数据。因此，通过加载为`(_, _)`来跳过它。

在你可以通过评估数据之前，你需要对其进行预处理：

`x_test`: 通过将每个像素除以255来对像素数据进行归一化（重新缩放）。这会用0到1之间的32位浮点数替换每个单字节整数像素。

`y_test`: 标签目前是标量（稀疏的）。如果你回顾一下`trainer/task.py`脚本中的`compile()`步骤，你会发现它是为稀疏标签编译的。所以你不需要做任何其他操作。

In [ ]:
(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

###执行模型评估

现在评估一下定制工作中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

### 图像数据的服务功能

要将图像传递给预测服务，您需要将压缩的（例如JPEG）图像字节编码为base64格式——这样在通过网络传输二进制数据时可以确保内容不被修改。由于部署的模型期望输入数据为原始（未压缩）字节，您需要确保base64编码的数据在传递给部署的模型之前被转换回原始字节。

为了解决这个问题，定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，以便将服务函数融合到底层模型中（而不是在CPU上游处理）。

当您发送预测或解释请求时，请求的内容会被base64解码为Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数会将`tf.string`预处理成原始（未压缩）的numpy字节（`preprocess_fn`）以符合模型的输入要求：
- `io.decode_jpeg`- 解压JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值转换为float32。
- `image.resize` - 调整图像大小以符合模型的输入形状。
- `resized / 255.0` - 将像素数据重新缩放（归一化）在0和1之间。

此时，数据可以传递给模型（`m_call`）。

#### XAI 签名

当服务函数与底层模型一起保存（`tf.saved_model.save`）时，您可以将服务函数的输入层指定为` serving_default` 签名。

对于XAI图像模型，您需要保存来自服务函数的另外两个签名：

- `xai_preprocess`：服务函数中的预处理函数。
- `xai_model`：用于调用模型的具体函数。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(local_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    local_model,
    model_path_to_deploy,
    signatures={
        "serving_default": serving_fn,
        # Required for XAI
        "xai_preprocess": preprocess_fn,
        "xai_model": m_call,
    },
)

获取服务功能签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获得模型输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务函数而不是模型。因此，您需要了解用于服务函数的输入层名称，以便在进行预测请求时稍后使用。

您还需要知道服务函数的输入和输出层的名称，以构建随后描述的解释元数据。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 说明规范

要获取预测的解释，您必须在将自定义模型上传到 Vertex AI 模型资源时启用解释功能并设置相应的设置。这些设置被称为解释元数据，包括：

- `参数`：这是用于对模型进行解释的可解释性算法的规范。您可以在以下选项中进行选择：
  - Shapley（注意：对于图像数据不建议使用，因为可能涉及长时间运行的操作）
  - XRAI
  - Integrated Gradients
- `元数据`：这是应用在您的自定义模型上的算法的规范。

#### 解释参数

查看解释算法设置的详细概述。

#### Shapley

为每个特征归因结果，考虑特征的不同排列。该方法提供了确切 Shapley 值的采样近似。

用途：
  - 在表格数据上进行分类和回归。

参数：

- `path_count`：这是算法处理的特征上路径数。获得确切的 Shapley 值需要 M！个路径，其中 M 是特征的数量。对于 CIFAR10 数据集，这将是 784（28*28）。

对于任何非平凡数量的特征而言，这太昂贵了。您可以将特征上路径的数量减少至 M * `path_count`。

#### Integrated Gradients

一种基于渐变的方法，用于有效地计算具有与 Shapley 值相同公理特性的特征归因。

用途：
  - 在表格数据上进行分类和回归。
  - 在图像数据上进行分类。

参数：

- `step_count`：这是用于近似剩余总和的步数。步数越多，积分近似越准确。一般规则是 50 步，但随着步数的增加，计算时间也会增加。

#### XRAI

基于集成梯度方法，XRAI 评估图像的重叠区域，以创建一个显著性地图，突出显示图像的相关区域而不是像素。

用途：
  - 在图像数据上进行分类。

参数：

- `step_count`：这是用于近似剩余总和的步数。步数越多，积分近似越准确。一般规则是 50 步，但随着步数的增加，计算时间也会增加。

在下一个代码单元格中，将变量 `XAI` 设置为您在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)

#### 解释元数据

现在，让我们更深入地了解解释元数据，其中包括：

- `outputs`：输出属性中的标量值 -- 要解释的内容。例如，在分类的概率输出\[0.1, 0.2, 0.7\]中，我们想要解释0.7。考虑以下公式，其中输出为 `y`，这就是我们要解释的内容。

    y = f(x)

考虑以下公式，其中输出为 `y` 和 `z`。由于我们只能对一个标量值进行归因，我们必须选择是要解释输出 `y` 还是 `z`。假设在这个例子中，模型是目标检测，y 和 z 分别是边界框和对象分类。您需要选择要解释的两个输出中的哪一个。

    y, z = f(x)

`outputs` 的字典格式如下：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 您分配给要解释的输出的易读名称。一个常见例子是“概率”。
 -  "output_tensor_name": 用于识别要解释的输出层的键/值字段。
 -  [layer]: 要解释的输出层。在单一任务模型中，比如表格回归器，它是模型中的最后一层（最顶层）。
</blockquote>

- `inputs`：归因的特征 -- 它们如何影响输出。考虑下面的公式，其中 `a` 和 `b` 是特征。我们必须选择要解释它们是如何影响输出的特征。假设这个模型部署用于A/B测试，其中 `a` 是用于预测的数据项，而 `b` 用于识别模型实例是A还是B。您将想要选择 `a`（或其子集）作为特征，而不选择 `b`，因为它不会对预测有所贡献。

    y = f(a,b)

`inputs` 的最小字典格式如下：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 您分配给要解释的输入的易读名称。一个常见例子是“特征”。
 -  "input_tensor_name": 用于识别特征归因的输入层的键/值字段。
 -  [layer]: 用于特征归因的输入层。在单输入张量模型中，它是模型中的第一层（最底层）。
</blockquote>

由于模型的输入是表格形式的，您可以指定以下两个额外字段作为报告/可视化辅助：

<blockquote>
 - "modality": "image": 表示字段值是图像数据。
</blockquote>

由于模型的输入是图像，您可以指定以下额外字段作为报告/可视化辅助：

<blockquote>
 - "modality": "image": 表示字段值是图像数据。
</blockquote>

In [ ]:
random_baseline = np.random.rand(32, 32, 3)
input_baselines = [{"number_vaue": x} for x in random_baseline]

INPUT_METADATA = {"input_tensor_name": CONCRETE_INPUT, "modality": "image"}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aiplatform.explain.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aiplatform.explain.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aiplatform.explain.ExplanationMetadata(
    inputs={"image": input_metadata}, outputs={"class": output_metadata}
)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到Vertex AI模型注册表，具有以下参数：

- `display_name`：模型资源的人类可读名称。
- `artifact`：经过训练的模型工件的Cloud Storage位置。
- `serving_container_image_uri`：用于提供的容器镜像。
- `sync`：是否异步或同步执行上传。
- `explanation_parameters`：配置解释模型预测的参数。
- `explanation_metadata`：描述用于解释的模型输入和输出的元数据。

如果`upload()`方法以异步方式运行，您可以随后使用`wait()`方法阻塞直到完成。

In [ ]:
model = aiplatform.Model.upload(
    display_name="cifar10",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

## 部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用`deploy()`方法，使用以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：指定流量分配在端点上前往该模型的百分比，格式为一个或多个键/值对的字典。
如果只有一个模型，则指定为 { "0": 100 }，其中"0"指代上传的模型以及100表示100%的流量。
如果端点上有现有的模型，需要分配流量，则使用model_id指定为 { "0": percent, model_id: percent, ... }，其中model_id是已部署端点的现有模型的模型 ID。百分比必须加起来等于100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：附加到工作者复制品的加速器数量。
- `starting_replica_count`：初始预留的计算实例数量。
- `max_replica_count`：可扩展到的最大计算实例数量。在本教程中，只有一个实例在预留。

In [ ]:
DEPLOYED_NAME = "cifar10"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

### 获取测试项

从数据集的测试（holdout）部分中使用一个例子作为测试项。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

### 准备请求内容
您将发送 CIFAR10 图像作为压缩的 JPG 图像，而不是原始未压缩的字节:

- `cv2.imwrite`: 使用 openCV 将未压缩的图像写入磁盘作为压缩的 JPEG 图像。
 - 将图像数据从 [0,1) 范围反归一化到 [0,255)。
 - 将32位浮点值转换为8位无符号整数。
- `tf.io.read_file`: 将压缩的 JPG 图像作为原始字节读入内存。
- `base64.b64encode`: 将原始字节编码为 base64 编码的字符串。

In [ ]:
cv2.imwrite("tmp.jpg", (test_image * 255).astype(np.uint8))

bytes = tf.io.read_file("tmp.jpg")
b64str = base64.b64encode(bytes.numpy()).decode("utf-8")

### 用解释做预测

现在您的模型资源已部署到终端资源，您可以通过向终端发送预测请求来获得在线解释。

#### 请求

每个实例的格式为：

    [{serving_input: {'b64': bytes}]

由于`explain()`方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目的列表发送。

#### 响应

`explain()`调用的响应是一个包含以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个实例的预测。
- `deployed_model_id`：部署模型资源的Vertex AI标识符。
- `explanations`：由可解释AI返回的特征归因。

In [ ]:
instances_list = [{serving_input: {"b64": b64str}}]

response = endpoint.explain(instances_list)
print(response)

### 可视化预测

预览图像及其预测类别，不包括解释。

In [ ]:
CLASSES = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

# Note: change the `ig_response` variable below if you didn't deploy an IG model
for prediction in response.predictions:
    label_index = np.argmax(prediction)
    class_name = CLASSES[label_index]
    confidence_score = prediction[label_index]
    print(
        "Predicted class: "
        + class_name
        + "\n"
        + "Confidence score: "
        + str(confidence_score)
    )

    image = base64.b64decode(b64str)
    image = BytesIO(image)
    img = mpimg.imread(image, format="JPG")

    plt.imshow(img, interpolation="nearest")
    plt.show()

### 用AI解释可视化图像

返回的图像显示了模型预测的顶部类别的解释。这意味着如果模型的预测有误，您看到突出显示的像素是用于*错误类别*的。例如，如果模型预测为"飞机"，而实际应该是"猫"，您可以看到模型将这幅图像分类为飞机的解释。

如果您部署了一个整合渐变模型，您可以可视化其特征归因。目前，从AI解释返回的突出显示的像素显示了对模型预测贡献最大的前60%的像素。在运行下面的单元格后看到的像素是最明显地标志着模型的预测的像素。

In [ ]:
for explanation in response.explanations:
    attributions = dict(explanation.attributions[0].feature_attributions)
    label_index = explanation.attributions[0].output_index[0]
    class_name = CLASSES[label_index]
    b64str = attributions["image"]["b64_jpeg"]
    image = base64.b64decode(b64str)
    image = io.BytesIO(image)
    img = mpimg.imread(image, format="JPG")

    plt.imshow(img, interpolation="nearest")
    plt.show()

## 卸载模型

当您完成使用模型进行预测后，请从终端点资源中卸载模型。此操作会取消所有计算资源并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的单个资源。

In [ ]:
# Delete the training job
job.delete()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the Cloud Storage bucket
delete_bucket = False  # Set True for deletion
if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

# Delete locally generated files
! rm -rf custom/
! rm custom.tar.gz
! rm tmp.jpg